In [2]:
from msgspec import json as mjs

In [3]:
byte_json = mjs.encode({'hello': 'world'})

In [4]:
byte_json

b'{"hello":"world"}'

In [5]:
encoder = mjs.Encoder()

In [6]:
new_byte_json = encoder.encode({'hello': 'json'})

In [7]:
new_byte_json

b'{"hello":"json"}'

In [8]:
dict_json = mjs.decode(byte_json)

In [9]:
dict_json

{'hello': 'world'}

In [10]:
decoder = mjs.Decoder()

In [11]:
decoder.decode(new_byte_json)

{'hello': 'json'}

In [15]:
from typing import Literal
import re
import msgspec

In [35]:
class ConfigSound(msgspec.Struct):
    type: Literal['alarm', 'signal', 'ambient']
    name: str

    def __post_init__(self):
        if self.type not in {'alarm', 'signal', 'ambient'}:
            raise ValueError('Sounds type has to be alarm, signal or ambient')
        
        # Regex pattern to ensure the 'name' consists only of alphanumeric characters, 
        # underscores, or hyphens, with no spaces or other special characters. The 
        # string must start and end with these characters, with at least one character in total.
        pattern = r'^[a-zA-Z0-9_-]+$'
        
        if not re.match(pattern, self.name):
            raise ValueError('Name must only contain alphanumeric characters, underscores, or hyphens')

class ConfigSoundsManager(msgspec.Struct):
    alarm: ConfigSound
    signal: ConfigSound
    ambient: ConfigSound
    
    def __post_init__(self):
        if self.alarm.type != 'alarm':
            raise ValueError('Alarm sound has to be of type alarm')
        if self.signal.type != 'signal':
            raise ValueError('Signal sound has to be of type signal')
        if self.ambient.type != 'ambient':
            raise ValueError('Ambient sound has to be of type ambient')

In [52]:
with open('config_focuskeeper_v1.yaml') as file:
    string_file = file.read()
    yaml_f = msgspec.yaml.decode(string_file)
    sounds = {key: ConfigSound(key, **value) for key, value in yaml_f['used_sounds'].items()}
    sounds_manager = ConfigSoundsManager(**sounds)

print(sounds_manager)
sounds_manager.ambient.name = 'Mario'
print(sounds_manager)

with open('config_focuskeeper_v1.yaml', 'w') as file:     
    file.write(msgspec.yaml.encode(sounds_manager))

ConfigSoundsManager(alarm=ConfigSound(type='alarm', name='Doorbell'), signal=ConfigSound(type='signal', name='Chime'), ambient=ConfigSound(type='ambient', name='Rain_Hard_Work_Hard'))
ConfigSoundsManager(alarm=ConfigSound(type='alarm', name='Doorbell'), signal=ConfigSound(type='signal', name='Chime'), ambient=ConfigSound(type='ambient', name='Mario'))


TypeError: write() argument must be str, not bytes